<img src="../../img/python-logo-no-text.svg"
     style="display:block;margin:auto;width:10%"/>
<br>
<div style="text-align:center; font-size:200%;">
  <b>Clean Code: Funktionen</b>
</div>
<br/>
<div style="text-align:center;">Dr. Matthias Hölzl</div>
<br/>
<div style="text-align:center;">module_230_clean_code/topic_140_a3_functions</div>


# Clean Code: Funktionen

Verpacke Sinnvolle Operationen als sorgfältig benannte Funktionen

- Besser lesbar
- Einfacher zu testen
- Wird eher wiederverwendet
- Fehler sind weniger wahrscheinlich


## Die 1. Clean Code Regel für Funktionen

- Funktionen sollten kurz sein
- Kürzer als man meint!
- Maximal 4 Zeilen!


## Lockere Regeln

(Aus den C++ Core Guidelines)

- Funktionen sollten auf einen Bildschirm passen
- Große Funktionen sollten in kleinere, zusammenhängende und benannte
  Funktionen aufgeteilt werden
- Funktionen mit einer bis fünf Zeilen sollten als normal angesehen werden


## Konzentration auf eine Aufgabe

- Funktionen sollten eine Aufgabe erfüllen ("do one thing")
- Sie sollten diese Aufgabe gut erfüllen
- Sie sollten nur diese Aufgabe erfüllen


## Abstraktionsebenen

Alles, was die Funktion in ihrem Rumpf tut, sollte eine (und nur eine)
Abstraktionsebene unterhalb der Funktion selbst sein.


## "Um zu"-Absätze: Kontrolle der Abstraktionsebenen

Um render_page_with_setups_and_teardowns durchzuführen

prüfen wir, ob die Seite eine Testseite ist und

wenn ja, binden wir Setup und Teardown ein.

In jedem Fall rendern wir die Seite in HTML.


## Die Step-Down-Regel

- Wir wollen, dass sich der Code wie eine Erzählung von oben nach unten liest
- Auf jede Funktion sollten die Funktionen eine Abstraktionsebene darunter
  folgen


## Mini-Workshop: Do one Thing

Die Funktion `handle_money_stuff()` macht mehr als eine Sache.

Teilen Sie sie in mehrere Funktionen auf, so dass jede nur eine Sache tut.
Stellen Sie sicher, dass
- jede Funktion ihre Aufgabe gut erfüllt und sich auf einer einzigen
  Abstraktionsebene befindet,
- alle Namen angemessen sind, und
- der Code leicht zu verstehen ist.

*Tipp:* Beginnen Sie damit, die Variablen gemäß den Kommentaren umzubenennen,
um den Rest der Arbeit zu vereinfachen.


Die Funktion `handle_money_stuff()` hat folgende Parameter:

- den Wochentag (`i_dow`, day of week),
- das Gehalt pro Tag (`f_spd`, salary per day),
- den Namen des Angestellten (`str_n`, name) und
- einen Liste der bisher gezahlten Gehälter (`lst_slrs`, salaries).

Das neue Gehalt wird an die Liste `lst_slrs` angehängt.

Die Funktion gibt die zu zahlende Steuer zurück.

In [ ]:
def handle_money_stuff(i_dow: int, f_spd: float, str_n: str, lst_slrs: list):
    # Get the day of week from the list of days.
    # We count Sunday as 1, Monday as 2, etc. but the work week starts on Monday.
    str_dow = lst_dns[i_dow - 1]
    # Compute the salary so far based on the day
    f_ssf = (i_dow - 1) * f_spd
    # The tax
    f_t = 0.0
    if f_ssf > 500.0 and f_ssf <= 1000.0:
        f_t = f_ssf * 0.05
    elif f_ssf > 1000.0 and f_ssf <= 2000.0:
        f_t = f_ssf * 0.1
    else:
        f_t = f_ssf * 0.15
    # Update salary based on the tax to pay
    f_ssf = f_ssf - f_t
    # Add the salary to the list of all salaries paid
    lst_slrs.append(f_ssf)
    print(f"{str_n} worked till {str_dow} and earned ${f_ssf} this week.")
    print(f"Their taxes were ${f_t}.")
    return f_t

In [ ]:
_salaries = [2345, 1234]
_result = handle_money_stuff(4, 200, "Joe", _salaries)
print(_result)

In [ ]:
assert _salaries == [2345, 1234, 570]
assert _result == 30

In [ ]:
_salaries = [2345, 1234]
_result = compute_salary_and_taxes_v1(4, 200, "Joe", _salaries)
print(_result)

In [ ]:
assert _salaries == [2345, 1234, 570]
assert _result == 30

In [ ]:
_salaries = [2345, 1234]
_result = compute_salary_and_taxes_v2(4, 200, "Joe", _salaries)
print(_result)

In [ ]:
assert _salaries == [2345, 1234, 570]
assert _result == 30


## Switch-Anweisungen und Abstraktion

Switch-Anweisungen führen oft Operationen auf der gleichen Abstraktionsebene aus.
(für "Subtypen" anstelle des ursprünglichen Typs).

"Subtypen" werden oft durch Typ-Tags unterschieden.

In [ ]:
from enum import IntEnum  # noqa: E402
from dataclasses import dataclass  # noqa: E402

In [ ]:
class EmployeeType(IntEnum):
    COMMISSIONED = 0
    HOURLY = 1
    SALARIED = 2

In [ ]:
@dataclass
class Money:
    amount_in_euros: float

    def __add__(self, rhs):
        return Money(self.amount_in_euros + rhs.amount_in_euros)

In [ ]:
print(Money(100.0) + Money(50.0))

In [ ]:
@dataclass
class EmployeeV1:
    type: EmployeeType

In [ ]:
def calculate_commissioned_pay(e: EmployeeV1):
    return Money(100.0)

In [ ]:
def calculate_hourly_pay(e: EmployeeV1):
    return Money(120.0)

In [ ]:
def calculate_salaried_pay(e: EmployeeV1):
    return Money(80.0)

In [ ]:
def calculate_pay(e: EmployeeV1):
    if e.type == EmployeeType.COMMISSIONED:
        return calculate_commissioned_pay(e)
    elif e.type == EmployeeType.HOURLY:
        return calculate_hourly_pay(e)
    elif e.type == EmployeeType.SALARIED:
        return calculate_salaried_pay(e)
    else:
        raise ValueError("No valid employee type.")

In [ ]:
e1 = EmployeeV1(type=EmployeeType.HOURLY)
e2 = EmployeeV1(EmployeeType.SALARIED)

In [ ]:
print(calculate_pay(e1))
print(calculate_pay(e2))


## Ersetze Switch-Anweisung durch Polymorphie

Es ist oft besser, switch-Anweisungen durch "echtes" Subtyping und
Polymorphismus zu ersetzen:

In [ ]:
from abc import ABC, abstractmethod  # noqa: E402

In [ ]:
class Employee(ABC):
    @abstractmethod
    def calculate_pay(self):
        ...

In [ ]:
@dataclass
class CommissionedEmployee(Employee):
    def calculate_pay(self):
        return Money(100.0)

In [ ]:
@dataclass
class HourlyEmployee(Employee):
    def calculate_pay(self):
        return Money(120.0)

In [ ]:
@dataclass
class SalariedEmployee(Employee):
    def calculate_pay(self):
        return Money(80.0)

In [ ]:
def create_employee(employee_type: EmployeeType):
    if employee_type == EmployeeType.COMMISSIONED:
        return CommissionedEmployee()
    elif employee_type == EmployeeType.HOURLY:
        return HourlyEmployee()
    elif employee_type == EmployeeType.SALARIED:
        return SalariedEmployee()
    else:
        raise ValueError("Not a valid employee type.")

In [ ]:
e1 = create_employee(EmployeeType.HOURLY)
e2 = create_employee(EmployeeType.SALARIED)

In [ ]:
print(e1.calculate_pay())
print(e2.calculate_pay())

In [ ]:
class EmployeeFactory:
    def create_employee(self, employee_type: EmployeeType):
        if employee_type == EmployeeType.COMMISSIONED:
            return CommissionedEmployee()
        elif employee_type == EmployeeType.HOURLY:
            return HourlyEmployee()
        elif employee_type == EmployeeType.SALARIED:
            return SalariedEmployee()
        else:
            raise ValueError("Not a valid employee type.")

In [ ]:
factory = EmployeeFactory()
e1 = factory.create_employee(EmployeeType.HOURLY)
e2 = factory.create_employee(EmployeeType.SALARIED)

In [ ]:
print(e1.calculate_pay())
print(e2.calculate_pay())


## Mini-Workshop: Ersetzen von Switch-Anweisungen

Strukturieren Sie den folgenden Code so um, dass nur noch bei der Erzeugung
der Objekte eine "switch-Anweisung" verwendet wird:

In [ ]:
from dataclasses import dataclass  # noqa: E402
from abc import ABC  # noqa: E402

In [ ]:
COMPUTER_TYPE_PC = 0
COMPUTER_TYPE_MAC = 1
COMPUTER_TYPE_CHROMEBOOK = 2

In [ ]:
@dataclass
class ComputerV1:
    computer_type: int

In [ ]:
def compile_code(computer: ComputerV1):
    if computer.computer_type == COMPUTER_TYPE_PC:
        print("Compiling code for PC.")
    elif computer.computer_type == COMPUTER_TYPE_MAC:
        print("Compiling code for Mac.")
    elif computer.computer_type == COMPUTER_TYPE_CHROMEBOOK:
        print("Compiling code for Chromebook.")
    else:
        raise ValueError(f"Don't know how to compile code for {computer}.")

In [ ]:
my_pc = ComputerV1(COMPUTER_TYPE_PC)
my_mac = ComputerV1(COMPUTER_TYPE_MAC)
my_chromebook = ComputerV1(COMPUTER_TYPE_CHROMEBOOK)

In [ ]:
compile_code(my_pc)
compile_code(my_mac)
compile_code(my_chromebook)


## Weitere Regeln für Funktionen

- Verwende beschreibende Namen
- Verwende wenige (oder keine) Argumente
- Verwende keine booleschen Argumente (Flag-Argumente)
- Vermeide versteckte Seiteneffekte


## Versteckte Seiteneffekte


```java
bool checkPassword(std::string userName, std::string password) {
    User& user = UserGateway.findByName(userName);
    if (user != User.NULL) {
        std::string codedPhrase = user.getPhraseEncodedByPassword();
        std::string phrase = cryptographer.decrypt(codedPhrase, password);
        if (phrase == "Valid Password") {
            session.initialize();
            return true;
        }
    }
    return false;
}
```


## Vermeide "Ausgabeargumente"

Python hat keine "echten" Ausgabeargumente. Aber Modifikation von Objekten hat
oft ähnliche Konsequenzen:

In [ ]:
class HitResultV1:
    pass

In [ ]:
class PlayerV1:
    def check_collision(self, obstacles, hit_result):
        # Complicated computation...
        hit_result.collision_occurred = True

In [ ]:
player = PlayerV1()
hit_result = HitResultV1()
player.check_collision([], hit_result)
if hit_result.collision_occurred:  # type: ignore
    print("Detected collision!")

In [ ]:
player = Player()
hit_result = player.check_collision([])
if hit_result.collision_occurred:
    print("Detected collision!")


## Command-Query Separation

In [ ]:
default_value = -1

In [ ]:
def bad_has_default_value() -> bool:
    global default_value
    if default_value >= 0:
        return True
    else:
        default_value = 123
        return False


## Fehlerbehandlung

Verwende Ausnahmen zur Fehlerbehandlung.

(Siehe `module_170_exceptions`.)


## DRY: Don't Repeat Yourself

- Versuche, duplizierten Code zu eliminieren.
  - Wiederholung bläht den Code auf
  - Wiederholung von Code erfordert mehrere Modifikationen für jede Änderung
- Aber: oft ist duplizierter Code mit anderem Code durchsetzt
- Berücksichtige den Bereich, in dem Sie den Code DRY halten!